# Etude - Piano Cover Generation

This notebook allows you to run the Etude pipeline directly from Google Drive.

## Setup

1.  **Mount Google Drive**: This allows the notebook to access your files.
2.  **Install Dependencies**: Installs the necessary libraries (Demucs, PyTorch, Madmom, etc.).
3.  **Download Models**: Downloads the pre-trained checkpoints required for inference.
4.  **Navigate to Project**: Changes the working directory to where you cloned/uploaded the Etude repository.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

# --- CONFIGURATION ---
# Change this path to match where you uploaded/cloned the Etude folder in your Drive.
# Example: '/content/drive/MyDrive/Projects/Etude'
PROJECT_PATH = '/content/drive/MyDrive/Etude'
# ---------------------

if not os.path.exists(PROJECT_PATH):
    print(f"WARNING: Path {PROJECT_PATH} does not exist. Please check your Drive structure.")
else:
    os.chdir(PROJECT_PATH)
    print(f"Current working directory: {os.getcwd()}")

## Install Dependencies
We need to install system libraries (ffmpeg) and Python packages.

**Note on Dependencies:**
The `synctoolbox` library requires `numpy<2.0` and `pandas<2.0`. We explicitly install these older versions to ensure compatibility. This might cause pip to show warnings about `google-colab` or other pre-installed packages being incompatible, but this is expected and necessary for the audio processing pipeline to work.

In [ ]:
!sudo apt-get update && sudo apt-get install -y ffmpeg

In [ ]:
# Ensure Cython is available for building madmom, but don't force a version.
!pip install Cython

In [ ]:
!pip install -r requirements.txt

In [ ]:
# Install Madmom from source with patching
# We clone the repo, patch the source code (fixing np.int/np.float issues), and then install.
# This ensures the Cython extensions are compiled with the correct NumPy types.

import os
if not os.path.exists('madmom'):
    !git clone https://github.com/CPJKU/madmom.git

# Run the patch script on the SOURCE directory
!python fix_madmom.py --path madmom

# Install from the patched source
!pip install ./madmom

In [ ]:
# Verify PyTorch versions
import torch
import torchvision
import torchaudio
print(f"Torch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")
print(f"Torchaudio version: {torchaudio.__version__}")

## Download Pre-trained Models
This step downloads the necessary model checkpoints and extracts them to the `checkpoints/` directory.

In [ ]:
import os

if not os.path.exists('checkpoints'):
    print("Downloading checkpoints...")
    !wget -O checkpoints.zip "https://github.com/Xiugapurin/Etude/releases/download/latest/checkpoints.zip"
    !unzip -q checkpoints.zip
    !rm checkpoints.zip
    print("Checkpoints downloaded and extracted.")
else:
    print("Checkpoints directory already exists. Skipping download.")

## Run Pipeline

You can now run the data preparation or inference scripts.

### Inference (Generate Piano Cover)
Use `infer.py` to generate a cover from a YouTube URL or local file.

In [ ]:
# Example: Generate a cover from a YouTube URL
# Replace the URL with your desired song.
!python infer.py --input "https://www.youtube.com/watch?v=dQw4w9WgXcQ" --output_name "my_cover"

### Resuming from a Crash
If the pipeline crashes (e.g., during Stage 2), you can resume from a specific stage using `--start-from`.

- `extract`: Start from beginning (Stage 1)
- `structuralize`: Start from Stage 2 (skips extraction)
- `decode`: Start from Stage 3 (skips extraction and beat detection)

In [ ]:
# Example: Resume from Stage 2 (Structuralize)
# !python infer.py --input "https://www.youtube.com/watch?v=dQw4w9WgXcQ" --output_name "my_cover" --start-from structuralize

### Data Preparation (Optional)
If you are training the model, use `prepare.py`.

In [ ]:
!python prepare.py --start-from download